# Partição dos dados

In [ ]:
!pip install pyspark

In [2]:
import pyspark
#conexão
from pyspark.sql import SparkSession 
from pyspark.sql.functions import max, count, col, asc, desc
#plots
import pandas as pd
import plotly.express as px

#### Transformando o arquivo em um dataframe

In [3]:
spark = SparkSession.builder\
      .master("local")\
      .appName("Particao")\
      .getOrCreate()

In [5]:
df = spark.read.csv("/content/2022-2023 Football Player Stats.csv", header= True, inferSchema = True, sep=";")

In [7]:
base = df.toPandas()
base['Pos'].value_counts()

DF      825
MF      608
FW      409
FWMF    244
MFFW    208
GK      164
DFMF    101
MFDF     62
DFFW     38
FWDF     30
Name: Pos, dtype: int64

#### categorizando as posições para realizar uma partição de dados que atendam aos nossos objetivos, separando eles por Defensor, Meio_Campo, Goleiro e Atacante

In [8]:
# o script abaixo foi criado para simplificar as posições dos jogadores
encoder = {'GK' : 'Goleiro',
#defensores
'DF':'Defensor','DFFW' : 'Defensor',
#meias
'MF': 'Meio_Campo', 'DFMF': 'Meio_Campo', 'MFFW': 'Meio_Campo', 'MFDF': 'Meio_Campo',
#atacantes
'FW': 'Atacante', 'FWDF' : 'Atacante', 'FWMF' : 'Atacante'}
Players = base.copy()
Players['funcao'] = Players.Pos.replace(encoder)

In [10]:
base['GPJ'] = base['Goals'] / base['MP']
base.head()

,Rk,Player,Nation,Pos,Squad,Comp,Age,Born,MP,Starts,...,Crs,TklW,PKwon,PKcon,OG,Recov,AerWon,AerLost,AerWon%,GPJ
0,1,Brenden Aaronson,USA,MFFW,Leeds United,Premier League,22,2000,20,19,...,2.54,0.51,0.0,0.0,0.00,4.86,0.34,1.19,22.2,0.05
1,2,Yunis Abdelhamid,MAR,DF,Reims,Ligue 1,35,1987,22,22,...,0.18,1.59,0.0,0.0,0.00,6.64,2.18,1.23,64.0,0.00
2,3,Himad Abdelli,FRA,MFFW,Angers,Ligue 1,23,1999,14,8,...,1.05,1.40,0.0,0.0,0.00,8.14,0.93,1.05,47.1,0.00
3,4,Salis Abdul Samed,GHA,MF,Lens,Ligue 1,22,2000,20,20,...,0.35,0.80,0.0,0.0,0.05,6.60,0.50,0.50,50.0,0.05
4,5,Laurent Abergel,FRA,MF,Lorient,Ligue 1,30,1993,15,15,...,0.23,2.02,0.0,0.0,0.00,6.51,0.31,0.39,44.4,0.00


In [11]:
Players.to_csv('/content/Players_2022-2023.csv')
df2 = spark.read.csv('/content/Players_2022-2023.csv', header = True, inferSchema= True)

Logo após criar uma coluna para simplificar as posições dos jogadores é realizada a partição de dados a partir da coluna funcao.

In [12]:
df2.write.partitionBy('funcao').mode('overwrite').save('/content/jogadores-posicao')